In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

from tqdm import tqdm

import warnings
warnings.filterwarnings('once')

In [3]:
df=pd.read_csv('../data/03_registrados_cod.csv', index_col=0)
df.head(2)

,año,sensacion_termica,humedad,viento,registrados,mes_map,estacion_map,dia_semana_map,clima_map,vacaciones_map,cod_SI
0,0,18.18125,80.5833,10.749882,654,0,0,0,1,0,0
1,0,17.68695,69.6087,16.652113,670,0,0,0,1,1,1


In [4]:
#  separamos nuestro dataframe en X e y

X = df.drop("registrados", axis = 1)
y = df["registrados"]

In [5]:
# dividimos nuestros datos en train y test para poder evaluar la validez de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [20]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error")

In [21]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [9, 10, 11, 12],
                         'max_features': [1, 2, 3],
                         'min_samples_leaf': [5, 15, 45],
                         'min_samples_split': [20, 30, 55]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [9]:
# al igual que el decision tree podemos sacar cual es nuestro mejor bosque
# En este caso,  nuestro mejor bosque es aquel que esta formado por arboles de 6 de profundidad, usa 4 variables y que tiene  un min_samples_leaf y un min_samples_split de 10. 

bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=10, max_features=3, min_samples_leaf=5,
                      min_samples_split=20)

In [10]:
# Calculamos sus métricas
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [11]:
# Con esta función calculamos las métricas y las convertimos en dataframe
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [12]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados
df_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
df_results

,MAE,MSE,RMSE,R2,set,modelo
0,491.251698,367469.370532,606.192519,0.829763,test,Random Forest
1,475.999755,392358.718736,626.385439,0.842621,train,Random Forest


In [17]:
#Concatenamos los resultados del df de Decision Tree y Random Forest 
df_resultados_DT = pd.read_csv('../data/05_dectre_registrados.csv', index_col= 0)
df_resultados_DT

,MAE,MSE,RMSE,R2,set,modelo
0,509.794521,442702.972603,665.359281,0.794909,test,Decission Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decission Tree I
0,621.447436,623736.521799,789.769917,0.711042,test,Decision tree II
1,504.566414,461827.338283,679.578795,0.814756,train,Decision tree II
0,573.929654,546012.145282,738.926346,0.747049,test,Decision tree III
1,550.614973,534654.432446,731.200679,0.785545,train,Decision tree III


In [18]:
df_todos_resultados = pd.concat([df_results, df_resultados_DT], axis = 0)
df_todos_resultados

,MAE,MSE,RMSE,R2,set,modelo
0,491.251698,367469.370532,606.192519,0.829763,test,Random Forest
1,475.999755,392358.718736,626.385439,0.842621,train,Random Forest
0,509.794521,442702.972603,665.359281,0.794909,test,Decission Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Decission Tree I
0,621.447436,623736.521799,789.769917,0.711042,test,Decision tree II
1,504.566414,461827.338283,679.578795,0.814756,train,Decision tree II
0,573.929654,546012.145282,738.926346,0.747049,test,Decision tree III
1,550.614973,534654.432446,731.200679,0.785545,train,Decision tree III


In [22]:
df_todos_resultados.to_csv('06_randForest_registrados.csv')